In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
# Для запуска графического ядра
import time


def time_matmul(x):
  start = time.time()
  for loop in range(100):
    tf.matmul(x, x)

  result = time.time()-start

  print("100 циклов: {:0.2f}ms".format(1000*result))

for _ in range(2):
    # исполнение CPU
    print(" CPU работает:")
    with tf.device("CPU:0"):
      x = tf.random.uniform([1000, 1000])
      assert x.device.endswith("CPU:0")
      time_matmul(x)

    # исполнение на GPU #0 , если доступен
    if tf.config.experimental.list_physical_devices("GPU"):
      print(" какой GPU:")
      with tf.device("GPU:0"): # или GPU:1 , для 2х GPU, GPU:2 для 3х  и т.д.
        x = tf.random.uniform([1000, 1000])
        assert x.device.endswith("GPU:0")
        time_matmul(x)

 CPU работает:
100 циклов: 357.33ms
 какой GPU:
100 циклов: 354.21ms
 CPU работает:
100 циклов: 357.33ms
 какой GPU:
100 циклов: 6.01ms


In [3]:
from tensorflow import keras 
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import boston_housing
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import scipy.stats as st
from scipy.stats import probplot, ks_2samp
from sklearn.preprocessing import normalize
from tensorflow.keras.optimizers import SGD, RMSprop,Adam
import re
from catboost import Pool, CatBoostRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, cross_val_score,RandomizedSearchCV
from tensorflow.keras.utils import plot_model

In [4]:
def coeff_determination(y_true, y_pred):
    from tensorflow.keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [5]:
# Загрузим данные с датасета 
(X_train,y_train),(X_test,y_test) = boston_housing.load_data()

In [6]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((404, 13), (404,), (102, 13), (102,))

In [12]:
# fig, axes = plt.subplots(4, 4, figsize=(15, 15))

# for num in range(X_train.shape[1]):
#     data = X_train[:,num].copy()
#     axes[num//4, num%4].hist(data, bins=30, color="blue")
#     axes[num//4, num%4].set_xlabel(f'f_{num}', fontsize=14)

In [13]:
# fig, axes = plt.subplots(4, 4, figsize=(20, 20))

# for i in range(X_train.shape[1]):
#     train_data, test_data = X_train[:,i].copy(), X_test[:,i].copy()
#     sns.kdeplot(train_data, ax=axes[i//4, i%4], color="blue", label="train")
#     sns.kdeplot(test_data, ax=axes[i//4, i%4], color="green", label="test")
   
#     axes[i//4, i%4].set_title(f'f_{i}', fontsize=14)

## Анализ распределения целевой переменной

In [14]:
# target = y_train
# fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# axes[0].set_title("Johnson SU", fontsize=14)
# sns.distplot(target, kde=False, color="blue", fit=st.johnsonsu, ax=axes[0])

# axes[1].set_title("Normal", fontsize=14)
# sns.distplot(target, kde=False, color="blue", fit=st.norm, ax=axes[1])

# axes[2].set_title("Log Normal", fontsize=14)
# sns.distplot(target, kde=False, color="blue", fit=st.lognorm, ax=axes[2])

In [15]:
# target = np.log(y_train)
# fig = plt.figure(figsize=(13, 5))
# plt.subplot(121)
# sns.distplot(target, bins=15, color="blue")
# plt.subplot(122)
# probplot(target, plot=plt)
# plt.show()

In [16]:
# target = y_train
# fig = plt.figure(figsize=(13, 5))
# plt.subplot(121)
# sns.distplot(target, bins=15, color="blue")
# plt.subplot(122)
# probplot(target, plot=plt)
# plt.show()

BASELAINE classic

In [13]:
param_grid_cb   = {'n_estimators':[50,100,200,300,400,500,700,800,600,1000],'l2_leaf_reg': [0.5,1,2,5,10,15,20,30,40,50],
                'max_depth': [2,3,4,5,6,7,8,9,10,12],'min_child_samples':[5,10,15,20,25,30,35,40,45,50],
                'max_bin':[5,10,25,30,35,40,45,50,55,60]}
cv_strategy = KFold(n_splits=5, shuffle=True, random_state=100)

In [14]:
%%time
model=RandomizedSearchCV(estimator=CatBoostRegressor(thread_count=15,early_stopping_rounds=90,verbose=False)\
                         ,cv=cv_strategy,  param_distributions = param_grid_cb,n_jobs = -1,n_iter = 50).fit(X_train,y_train)
print( model.best_params_)
valid_predict = model.best_estimator_.predict(X_test)
r2_score(y_test,valid_predict)

{'n_estimators': 1000, 'min_child_samples': 20, 'max_depth': 4, 'max_bin': 45, 'l2_leaf_reg': 15}
Wall time: 1min 8s


0.8700559993340975

### Нормализуем данные

In [15]:
# X_train_w=normalize(X_train)
# X_test_w = normalize(X_test)
# y_train_w = np.log(y_train)
# y_test_w = np.log(y_test)

### Сделаем тестовую модель, что бы проверить как работает и с какими данными лучше работать

In [16]:
# # Заполняем модель, формируя граф передачи тензоров
# x_input = Input(shape=(13))
# x = Flatten()(x_input)
# x = Dense(200, activation='sigmoid')(x)
# x = Dense(300, activation='relu')(x)
# x_output = Dense(1)(x)
# model=Model(x_input,x_output,name = '22')

In [7]:
opt_adam = keras.optimizers.Adam()
loss=tf.keras.losses.MeanSquaredError()
metric= coeff_determination


In [18]:
# model.compile(optimizer=opt_adam,
#               loss=loss,
#               metrics=[metric])

In [19]:
# ww = model.get_weights()

In [18]:
# # Данныее без изменений
# model.set_weights(ww)
# hh_1 = model.fit(X_train,y_train,epochs=10,validation_data=(X_test, y_test),verbose=1)
# model.evaluate(X_test,y_test)

In [8]:
N=200
models=[]
x_input = Input(shape=(13))
levels=np.arange(2,10)
num_n = np.arange(130,1001,13)
fun = ['relu','sigmoid','relu','sigmoid','relu','relu','tanh','relu','relu']

for _ in range(N):
    level = np.random.choice(levels)
    x = Flatten()(x_input)
    my_level = []
    for l in range(level):
        n = np.random.choice(num_n)
        f = np.random.choice(fun)
        x=Dense(n,activation=f)(x)
        my_level.append((n,f))
    x_output = Dense(1)(x)   
    name=f'{level}_{my_level}'
    name =re.sub("[\[\]()'',]",'',name)
    name=re.sub(" ",'_',name)
    model=Model(x_input,x_output,name=name)
    models.append((model,model.get_weights()))

In [9]:
for i in range(5):
    print(models[i][0].name)


7_650_sigmoid_559_sigmoid_338_relu_780_relu_910_relu_806_relu_455_relu
7_845_sigmoid_234_sigmoid_624_sigmoid_377_sigmoid_390_relu_416_relu_949_relu
7_143_relu_351_tanh_663_tanh_403_relu_819_relu_273_relu_611_sigmoid
9_923_relu_598_sigmoid_429_relu_585_relu_585_tanh_468_relu_364_relu_624_relu_182_tanh
8_143_relu_520_relu_507_relu_650_sigmoid_923_sigmoid_507_relu_351_sigmoid_650_relu


In [10]:
len(models)

200

In [11]:
%%time
n_test = 10
result = pd.DataFrame()
for model in models:
    for opt in [Adam()]:
        for epochs in [75,150]:
            model[0].compile(optimizer=opt,
                          loss=tf.keras.losses.MeanSquaredError(),
                          metrics=[coeff_determination])
            testing = []
            for _ in range(n_test):
                model[0].set_weights(model[1])
                hh = model[0].fit(X_train,y_train,epochs=epochs,\
                              validation_data=(X_test, y_test),verbose=0,batch_size=1000)
                answer = hh.history['val_coeff_determination'][-1]
                testing.append(answer)
            testing = testing = np.array(testing)
            d={
                'name':model[0].name,
                'epoch': epochs,
                'opt':opt,
                'res_min':testing.min(),
                'res_max':testing.max(),
                'res_mean':testing.mean(),
                'res_std':testing.std(),
               }

            result = result.append(d,ignore_index=True)
            


Wall time: 2h 2min 20s


In [12]:
result.sort_values('res_mean',ascending=False,inplace=True)
result.head(10)

,epoch,name,opt,res_max,res_mean,res_min,res_std
177,150.0,2_923_tanh_988_relu,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.732828,0.712879,0.676502,0.016364
169,150.0,2_741_tanh_975_relu,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.734671,0.712604,0.681297,0.017437
149,150.0,4_338_sigmoid_650_relu_806_relu_325_relu,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.717172,0.691426,0.614096,0.030923
237,150.0,3_546_sigmoid_806_relu_832_relu,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.727752,0.688071,0.619649,0.037749
21,150.0,3_845_sigmoid_884_relu_728_relu,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.714231,0.678858,0.632282,0.021623
231,150.0,2_572_relu_689_relu,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.701270,0.674727,0.643448,0.014990
397,150.0,4_806_sigmoid_143_relu_403_relu_936_relu,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.735177,0.664312,0.501252,0.060652
263,150.0,2_546_relu_455_relu,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.665814,0.652638,0.618392,0.013425
85,150.0,2_767_sigmoid_637_relu,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.694383,0.646847,0.577424,0.043431
69,150.0,6_871_sigmoid_650_relu_468_relu_247_relu_884_r...,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.686834,0.646529,0.586652,0.036073


### Возмем первые топ 10 моделей и проверим их на устойчивость

In [13]:
def name_model(name):
    for m in models:
        if m[0].name == name:
            return m
        
best_models=[]
for i in range(10):
    my_iter = result.iloc[i]
    best_models.append((name_model(my_iter['name']),my_iter['epoch'],my_iter['opt']))
    

In [14]:
best_models[0][0][0].name

'2_923_tanh_988_relu'

In [15]:
%%time
test_num = 50
best_result = pd.DataFrame()
for m in best_models:
    testing=[]
    for _ in range (test_num):
        
        m[0][0].set_weights(m[0][1])
        hh = m[0][0].fit(X_train,y_train,epochs=int(m[1]),\
                          validation_data=(X_test, y_test),verbose=0,batch_size=1000)
        answer = hh.history['val_coeff_determination'][-1]
        testing.append(answer)
    testing = np.array(testing)
    d={
        'name':m[0][0].name,
        'epoch': m[1],
        'opt':m[2],
        'res_min':testing.min(),
        'res_max':testing.max(),
        'res_mean':testing.mean(),
        'res_std':testing.std(),
        'res_median':np.median(testing)
    }
    best_result = best_result.append(d,ignore_index=True)

Wall time: 17min 49s


In [16]:
best_result.sort_values('res_mean',ascending=False,inplace=True)
best_result

,epoch,name,opt,res_max,res_mean,res_median,res_min,res_std
1,150.0,2_741_tanh_975_relu,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.736022,0.711259,0.711237,0.678766,0.011835
0,150.0,2_923_tanh_988_relu,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.742504,0.707974,0.709502,0.665977,0.020682
3,150.0,3_546_sigmoid_806_relu_832_relu,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.735497,0.682744,0.690178,0.598915,0.035085
2,150.0,4_338_sigmoid_650_relu_806_relu_325_relu,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.734308,0.676304,0.681211,0.554438,0.040720
5,150.0,2_572_relu_689_relu,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.703165,0.663903,0.664882,0.613228,0.020126
4,150.0,3_845_sigmoid_884_relu_728_relu,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.716705,0.659665,0.666879,0.582257,0.037770
7,150.0,2_546_relu_455_relu,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.688745,0.656539,0.655483,0.615488,0.012230
8,150.0,2_767_sigmoid_637_relu,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.692105,0.617783,0.622787,0.538721,0.032760
6,150.0,4_806_sigmoid_143_relu_403_relu_936_relu,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.735179,0.605477,0.613650,0.439480,0.066799
9,150.0,6_871_sigmoid_650_relu_468_relu_247_relu_884_r...,<tensorflow.python.keras.optimizer_v2.adam.Ada...,0.676239,0.496312,0.542439,-0.095041,0.181059


d 
### Отберем наиболее лучшие три модели  и еще раз попробуем разное количество эпох для них





In [17]:
best_models_full_data=[]
for i in range(3):
    my_iter = best_result.iloc[i]
    print(my_iter['name'])
    best_models_full_data.append(name_model(my_iter['name']))
    

2_741_tanh_975_relu
2_923_tanh_988_relu
3_546_sigmoid_806_relu_832_relu


In [18]:
%%time
test_num = 50
best_result_full_data = pd.DataFrame()
for m in best_models_full_data:
    for ep in [300,500]:
        testing=[]
        for _ in range(test_num):
            m[0].set_weights(m[1])
            hh = m[0].fit(X_train,y_train,epochs=ep,\
                              validation_data=(X_test, y_test),verbose=0,batch_size=1000)
            answer = hh.history['val_coeff_determination'][-1]
            testing.append(answer)
        testing = np.array(testing)
        d={
            'name':m[0].name,
            'epoch': ep,
            'res_min':testing.min(),
            'res_max':testing.max(),
            'res_mean':testing.mean(),
            'res_std':testing.std(),
            'res_median':np.median(testing)
        }
        best_result_full_data = best_result_full_data.append(d,ignore_index=True)
best_result_full_data.sort_values('res_mean',ascending=False,inplace=True)
best_result_full_data  

Wall time: 27min 51s


,epoch,name,res_max,res_mean,res_median,res_min,res_std
2,300.0,2_923_tanh_988_relu,0.766633,0.748996,0.749634,0.710645,0.009344
0,300.0,2_741_tanh_975_relu,0.754453,0.735000,0.735758,0.720485,0.007800
3,500.0,2_923_tanh_988_relu,0.755957,0.732467,0.731949,0.709599,0.012317
5,500.0,3_546_sigmoid_806_relu_832_relu,0.769193,0.727631,0.724691,0.689816,0.018321
4,300.0,3_546_sigmoid_806_relu_832_relu,0.754642,0.724166,0.726923,0.686380,0.017218
1,500.0,2_741_tanh_975_relu,0.749037,0.719991,0.719589,0.696104,0.012885


### сделаем ансабль

In [19]:
a = best_models_full_data[0][0]
b = best_models_full_data[1][0]
c = best_models_full_data[2][0]

out_all = keras.layers.Average()([a.output, b.output ,c.output])
model_ansabl = Model(a.input,out_all,name = 'ansbl_full_data')

model_ansabl.compile(optimizer=opt_adam,
              loss=loss,
              metrics=[metric])
ww=model_ansabl.get_weights()

avarage

In [20]:
%%time
test_num = 50
result_ansamble = pd.DataFrame()
for ep in [300,100,500]:
    testing=[]
    for _ in range(test_num):
        model_ansabl.set_weights(ww)
        hh = model_ansabl.fit(X_train,y_train,epochs=ep,\
                          validation_data=(X_test, y_test),verbose=0,batch_size=1000)
        answer = hh.history['val_coeff_determination'][-1]
        testing.append(answer)
    testing = np.array(testing)
    d={
        'name':model_ansabl.name,
        'epoch': ep,
        'type_ansabl':'avg',
        'res_min':testing.min(),
        'res_max':testing.max(),
        'res_mean':testing.mean(),
        'res_std':testing.std(),
        'res_median':np.median(testing)
    }
    result_ansamble = result_ansamble.append(d,ignore_index=True)
result_ansamble.sort_values('res_mean',ascending=False,inplace=True)
result_ansamble

Wall time: 13min 59s


,epoch,name,res_max,res_mean,res_median,res_min,res_std,type_ansabl
2,500.0,ansbl_full_data,0.757275,0.745804,0.748409,0.727497,0.008303,avg
0,300.0,ansbl_full_data,0.754115,0.738271,0.741913,0.712185,0.008984,avg
1,100.0,ansbl_full_data,0.736865,0.727726,0.732959,0.680886,0.010904,avg


Maximum

In [21]:
a = best_models_full_data[0][0]
b = best_models_full_data[1][0]
c = best_models_full_data[2][0]

out_all = keras.layers.Maximum()([a.output, b.output ,c.output])
model_ansabl = Model(a.input,out_all,name = 'ansbl_full_data')

model_ansabl.compile(optimizer=opt_adam,
              loss=loss,
              metrics=[metric])
ww=model_ansabl.get_weights()

In [22]:
%%time
test_num = 10
# result_ansamble = pd.DataFrame()
for ep in [300,100,500]:
    testing=[]
    for _ in range(test_num):
        model_ansabl.set_weights(ww)
        hh = model_ansabl.fit(X_train,y_train,epochs=ep,\
                          validation_data=(X_test, y_test),verbose=0,batch_size=1000)
        answer = hh.history['val_coeff_determination'][-1]
        testing.append(answer)
    testing = np.array(testing)
    d={
        'name':model_ansabl.name,
        'epoch': ep,
        'type_ansabl':'max',
        'res_min':testing.min(),
        'res_max':testing.max(),
        'res_mean':testing.mean(),
        'res_std':testing.std(),
        'res_median':np.median(testing)
    }
    result_ansamble = result_ansamble.append(d,ignore_index=True)
result_ansamble.sort_values('res_mean',ascending=False,inplace=True)
result_ansamble

Wall time: 2min 51s


,epoch,name,res_max,res_mean,res_median,res_min,res_std,type_ansabl
0,500.0,ansbl_full_data,0.757275,0.745804,0.748409,0.727497,0.008303,avg
1,300.0,ansbl_full_data,0.754115,0.738271,0.741913,0.712185,0.008984,avg
3,300.0,ansbl_full_data,0.757872,0.736332,0.736441,0.710021,0.011882,max
4,100.0,ansbl_full_data,0.751476,0.734479,0.731571,0.722766,0.009187,max
2,100.0,ansbl_full_data,0.736865,0.727726,0.732959,0.680886,0.010904,avg
5,500.0,ansbl_full_data,0.742399,0.716461,0.715696,0.686200,0.014937,max


Minimum

In [23]:
a = best_models_full_data[0][0]
b = best_models_full_data[1][0]
c = best_models_full_data[2][0]

out_all = keras.layers.Minimum()([a.output, b.output ,c.output])
model_ansabl = Model(a.input,out_all,name = 'ansbl_full_data')

model_ansabl.compile(optimizer=opt_adam,
              loss=loss,
              metrics=[metric])
ww=model_ansabl.get_weights()

In [24]:
%%time
test_num = 10
# result_ansamble = pd.DataFrame()
for ep in [300,100,500]:
    testing=[]
    for _ in range(test_num):
        model_ansabl.set_weights(ww)
        hh = model_ansabl.fit(X_train,y_train,epochs=ep,\
                          validation_data=(X_test, y_test),verbose=0,batch_size=1000)
        answer = hh.history['val_coeff_determination'][-1]
        testing.append(answer)
    testing = np.array(testing)
    d={
        'name':model_ansabl.name,
        'epoch': ep,
        'type_ansabl':'min',
        'res_min':testing.min(),
        'res_max':testing.max(),
        'res_mean':testing.mean(),
        'res_std':testing.std(),
        'res_median':np.median(testing)
    }
    result_ansamble = result_ansamble.append(d,ignore_index=True)
result_ansamble.sort_values('res_mean',ascending=False,inplace=True)
result_ansamble

Wall time: 2min 53s


,epoch,name,res_max,res_mean,res_median,res_min,res_std,type_ansabl
0,500.0,ansbl_full_data,0.757275,0.745804,0.748409,0.727497,0.008303,avg
1,300.0,ansbl_full_data,0.754115,0.738271,0.741913,0.712185,0.008984,avg
2,300.0,ansbl_full_data,0.757872,0.736332,0.736441,0.710021,0.011882,max
3,100.0,ansbl_full_data,0.751476,0.734479,0.731571,0.722766,0.009187,max
4,100.0,ansbl_full_data,0.736865,0.727726,0.732959,0.680886,0.010904,avg
5,500.0,ansbl_full_data,0.742399,0.716461,0.715696,0.686200,0.014937,max
6,300.0,ansbl_full_data,0.718372,0.706217,0.705524,0.696921,0.007537,min
8,500.0,ansbl_full_data,0.725292,0.702595,0.703576,0.685902,0.010543,min
7,100.0,ansbl_full_data,0.720991,0.676484,0.709480,0.384851,0.097655,min
